In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#1. Data Loading
import tensorflow as tf

tf.reset_default_graph()

data = pd.read_csv("./Data/Digit/train.csv",sep = ",")
test = pd.read_csv("./Data/Digit/test.csv",sep = ",")
num_of_train = int(data.shape[0] * 0.7)
train_data = data.loc[: num_of_train,"pixel0":]
# train_label =  tf.one_hot(train_data["label"].values,10)
train_label = pd.get_dummies(data.loc[: num_of_train,"label"])
test_data = data.loc[num_of_train:,"pixel0":]
# test_label =  tf.one_hot(test_data["label"].values,10)
test_label = pd.get_dummies(data.loc[num_of_train:,"label"])


# sess = tf.Session()
# sess.run(label_data[[0]])


#2. Placeholder
import tensorflow as tf
X = tf.placeholder(shape = [None,784], dtype = tf.float32)
#The number of columns is 42000
Y = tf.placeholder(shape = [None,10], dtype = tf.float32)
keep_rate = tf.placeholder(dtype = tf.float32)

#3. Convolution Layer
#Replace the image data with a squared array. 
X_img = tf.reshape(X, shape = [-1,28,28,1])
L1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3]
                     ,padding = "SAME", strides = 1, activation = tf.nn.relu)

#MaxPooling
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size = [2,2]
                            ,padding = "SAME", strides = 2 )

#Filter, Convolution, ReLU
L2 = tf.layers.conv2d(inputs = L1, filters = 64, kernel_size = [3,3]
                     ,padding = "SAME", strides = 1, activation = tf.nn.relu)

#MaxPooling
L2 = tf.layers.max_pooling2d(inputs = L2, pool_size = [2,2]
                            ,padding = "SAME", strides = 2 )


#4. FC(Neural Network)
L2 = tf.reshape(L2, shape = [-1,7*7*64])

#5. Weight & bias
W2 = tf.get_variable("weight2", shape= [7*7*64,256],initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name ="bias2")

_layer1 = tf.nn.relu(tf.matmul(L2,W2) + b2)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_rate)

W3 = tf.get_variable("weight3", shape = [256,256],initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256]), name = "bias3")

_layer2 = tf.nn.relu(tf.matmul(layer1,W3) + b3)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_rate)

W4 = tf.get_variable("weight4", shape = [256,10],initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([10]), name = "bias4")

#6. Hypothesis
H = tf.matmul(layer2, W4) + b4

#7. Cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = H, labels= Y))

#8. Train
train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

#9. Session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#10.학습
num_of_epoch = 10
batch_size = 100


for step in range(num_of_epoch):
    num_of_iter = int(train_data.shape[0]/batch_size)
    for i in range(num_of_iter):
        batch_x = train_data.iloc[i*(batch_size):i*(batch_size)+(batch_size)]
        batch_y = train_label.iloc[i*(batch_size):i*(batch_size)+(batch_size)]
        _, cost_val = sess.run([train, cost], feed_dict = {X: batch_x, Y: batch_y, keep_rate: 0.5})

    print("Cost : {}".format(cost_val))

predict = tf.argmax(H,1)
correct = tf.equal(predict,tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))

print("정확도 :{}".format(sess.run(accuracy, feed_dict = {X: test_data, Y: test_label, keep_rate:1})))

##11. 파일 생성
batch_size = 100

predict = tf.argmax(H,1)
enter = []

num_of_iter = int(test.shape[0]/batch_size)
for i in range(num_of_iter):
    batch_x = test.iloc[i*(batch_size):i*(batch_size)+(batch_size)]
    result = sess.run(predict, feed_dict = {X: batch_x, keep_rate:1})
    
    for j in range(len(result)):
        enter.append(result[j])

data3 = pd.read_csv("./Data/Digit/sample_submission.csv",sep=",")
data3["Label"] = enter

data3.to_csv("./Data/Digit/submission.csv",index = False)


Cost : 0.31484293937683105
Cost : 0.09023290872573853
Cost : 0.028296081349253654
Cost : 0.012673807330429554
Cost : 0.026862576603889465
Cost : 0.03918340057134628
Cost : 0.006506930571049452
Cost : 0.00678820488974452
Cost : 0.012393884360790253
Cost : 0.06254761666059494
정확도 :0.983890175819397
